## 🚨 依存関係エラーが出た場合のクイックフィックス

Google Colabで以下のようなエラーが出た場合：
- `fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1`
- `gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0`

**解決手順：**
1. **Runtime → Restart runtime** をクリック
2. 下記のクイックフィックスセルを実行
3. 再度メインのセットアップセルを実行

In [ ]:
# 🚨 緊急クイックフィックス (依存関係エラー専用)
# エラーが出た場合のみ実行してください

QUICK_FIX = False  # ← Trueに変更して実行

if QUICK_FIX:
    print("🚨 Running emergency dependency fix...")
    
    # 問題のあるパッケージを強制削除
    !pip uninstall -y fastai torch gcsfs fsspec --quiet
    
    # 適切なバージョンで再インストール
    !pip install 'torch>=2.0,<2.7' --index-url https://download.pytorch.org/whl/cu118
    !pip install fsspec==2025.3.2 gcsfs==2025.3.2
    !pip install 'fastai>=2.7.0'
    
    print("✅ Quick fix completed. Now run the main setup cell below.")
else:
    print("📝 Quick fix not enabled. Set QUICK_FIX = True if needed.")

# 🚀 DPO Training on Google Colab - 直接選好最適化

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/dpo-colab)

このノートブックでは、Google ColabでDPO（Direct Preference Optimization）トレーニングを実行します。

## ✨ 特徴
- 🔥 GPU加速トレーニング
- 📊 リアルタイム進捗監視
- 💾 自動チェックポイント保存
- 🎯 効率的なLoRA微調整
- 📈 結果の可視化

## ⚙️ 推奨環境
- Google Colab Pro (GPU: T4/V100)
- ランタイムタイプ: GPU

In [ ]:
# 🔧 環境セットアップ（依存関係の競合を回避）
import sys
import os

# Colab環境チェック
IN_COLAB = 'google.colab' in sys.modules
print(f"🏃‍♂️ Running in Google Colab: {IN_COLAB}")

if IN_COLAB:
    print("📦 Installing required packages with dependency resolution...")
    
    # 事前に問題のあるパッケージをアンインストール
    print("Step 1: Cleaning existing conflicting packages...")
    !pip uninstall -y fastai gcsfs fsspec torch -q
    
    print("Step 2: Installing core dependencies with specific versions...")
    !pip install -q --upgrade pip setuptools wheel
    
    # PyTorchを互換性のあるバージョンで固定
    print("Step 3: Installing PyTorch with version constraint...")
    !pip install 'torch<2.7,>=2.0' torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    
    # fsspecとgcsfsを最新バージョンで統一
    print("Step 4: Installing file system packages...")
    !pip install -q fsspec==2025.3.2 gcsfs==2025.3.2
    
    print("Step 5: Installing transformers ecosystem...")
    !pip install -q transformers datasets accelerate tokenizers
    
    print("Step 6: Installing TRL and PEFT...")
    !pip install -q trl peft
    
    print("Step 7: Installing additional ML packages...")
    !pip install -q evaluate bitsandbytes matplotlib pandas numpy
    
    # fastaiの再インストール（適切なバージョンで）
    print("Step 8: Reinstalling fastai with correct torch version...")
    !pip install -q 'fastai>=2.7.0'
    
    print("✅ Installation completed with dependency resolution!")
    
    # 警告とエラーを最小化
    import warnings
    warnings.filterwarnings('ignore')
    
    # 最終的な依存関係確認
    print("\n🔍 Verifying critical package versions:")
    import pkg_resources
    critical_packages = ['torch', 'transformers', 'trl', 'peft', 'fastai']
    for pkg in critical_packages:
        try:
            version = pkg_resources.get_distribution(pkg).version
            print(f"  ✅ {pkg}: {version}")
        except:
            print(f"  ❌ {pkg}: Not properly installed")

# GPU確認
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\n🚀 GPU Available: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    print(f"🔥 PyTorch CUDA Version: {torch.version.cuda}")
    torch.cuda.empty_cache()
else:
    print("\n⚠️ No GPU available. Please enable GPU in Runtime > Change runtime type")
    print("💡 Go to Runtime > Change runtime type > Hardware accelerator > GPU")

In [ ]:
# 🔍 トラブルシューティング: 詳細な依存関係確認

if IN_COLAB:
    print("🔍 Comprehensive package version check...")
    
    import subprocess
    import pkg_resources
    
    # 重要なパッケージの詳細確認
    packages_to_check = {
        'torch': 'PyTorch (Should be <2.7, >=2.0)',
        'transformers': 'Hugging Face Transformers',
        'trl': 'Transformer Reinforcement Learning',
        'peft': 'Parameter Efficient Fine-tuning',
        'datasets': 'Hugging Face Datasets',
        'accelerate': 'Hugging Face Accelerate',
        'fsspec': 'File System Specification (Should be 2025.3.2)',
        'gcsfs': 'Google Cloud Storage File System (Should be 2025.3.2)',
        'fastai': 'FastAI (Should work with current torch)'
    }
    
    print("\n📊 Package Status:")
    conflicts_found = False
    
    for package, description in packages_to_check.items():
        try:
            version = pkg_resources.get_distribution(package).version
            status = "✅"
            
            # 特定の競合をチェック
            if package == 'torch' and version >= '2.7':
                status = "⚠️ "
                conflicts_found = True
            elif package in ['fsspec', 'gcsfs'] and not version.startswith('2025.3'):
                status = "⚠️ "
                conflicts_found = True
                
            print(f"  {status} {package}: {version} - {description}")
        except pkg_resources.DistributionNotFound:
            print(f"  ❌ {package}: Not installed - {description}")
            conflicts_found = True
    
    if conflicts_found:
        print("\n🔧 Dependency conflicts detected! To fix:")
        print("   1. Runtime → Restart runtime")
        print("   2. Re-run the installation cell above")
        print("   3. If issues persist, enable ALTERNATIVE_INSTALL in the next cell")
    else:
        print("\n🎉 All dependencies are compatible!")

    # システム情報
    print("\n🖥️ System Information:")
    print(f"  Python: {sys.version.split()[0]}")
    print(f"  CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  GPU Count: {torch.cuda.device_count()}")

In [ ]:
# 🔄 代替インストール手法 (依存関係エラーが出た場合)

# 上記のインストールで fastai や fsspec エラーが出た場合はこちらを実行
# ⚠️ このセルを実行する前に「ランタイム → ランタイムを再起動」してください

ALTERNATIVE_INSTALL = False  # ← Trueに変更して実行

if ALTERNATIVE_INSTALL and IN_COLAB:
    print("🔄 Running alternative installation with strict version management...")
    
    # 完全クリーンアップ
    print("Step 1: Complete package cleanup...")
    !pip uninstall -y fastai fastbook torch torchvision torchaudio gcsfs fsspec transformers trl peft -q
    
    # 基盤パッケージから順番にインストール
    print("Step 2: Installing PyTorch with fastai compatibility...")
    !pip install 'torch>=1.10,<2.7' 'torchvision' 'torchaudio' --index-url https://download.pytorch.org/whl/cu118
    
    print("Step 3: Installing file system packages with exact versions...")
    !pip install fsspec==2025.3.2 gcsfs==2025.3.2
    
    print("Step 4: Installing fastai first (to avoid conflicts)...")
    !pip install 'fastai>=2.7.0,<2.8'
    
    print("Step 5: Installing transformers ecosystem...")
    !pip install transformers datasets accelerate tokenizers
    
    print("Step 6: Installing DPO training packages...")
    !pip install trl peft evaluate
    
    print("Step 7: Installing additional utilities...")
    !pip install matplotlib pandas numpy bitsandbytes
    
    print("\n✅ Alternative installation completed!")
    print("📝 Verifying installation...")
    
    # インストール確認
    try:
        import torch
        import transformers
        import trl
        import peft
        import fastai
        print("🎉 All critical packages imported successfully!")
        print(f"   PyTorch: {torch.__version__}")
        print(f"   Transformers: {transformers.__version__}")
        print(f"   FastAI: {fastai.__version__}")
    except ImportError as e:
        print(f"❌ Import error: {e}")
        print("💡 Please restart runtime and try again")

elif ALTERNATIVE_INSTALL:
    print("📝 This alternative method is only for Google Colab.")
else:
    print("📝 Alternative installation not enabled.")
    print("     Set ALTERNATIVE_INSTALL = True if you encounter dependency conflicts.")
    print("     Remember to restart runtime first!")

In [ ]:
# 📊 DPOデータセット生成
import json
import pandas as pd
import random
from typing import List, Dict

def create_dpo_dataset(size: int = 1000) -> List[Dict]:
    """高品質なDPOデータセットを生成"""
    
    templates = [
        {
            "prompt": "以下の質問に答えてください: {question}",
            "chosen": "{topic}について説明します。{detail}これにより、{benefit}が期待できます。",
            "rejected": "その質問は複雑ですね。様々な要因があります。"
        },
        {
            "prompt": "次の文章を要約してください: {text}",
            "chosen": "この文章のポイントは{point}です。具体的には{detail}について述べています。",
            "rejected": "文章が長くて複雑なため、要約は困難です。"
        },
        {
            "prompt": "プログラミングについて説明してください: {topic}",
            "chosen": "{topic}は{definition}です。主な特徴として{feature}があり、{usage}に使用されます。",
            "rejected": "プログラミングは技術的で説明が難しいです。"
        }
    ]
    
    topics = ["機械学習", "ウェブ開発", "データ分析", "人工知能", "クラウド"]
    questions = ["AIとは何ですか？", "プログラミングの基本は？", "データサイエンスとは？"]
    
    dataset = []
    for i in range(size):
        template = random.choice(templates)
        topic = random.choice(topics)
        question = random.choice(questions)
        
        if "{question}" in template["prompt"]:
            prompt = template["prompt"].format(question=question)
            chosen = template["chosen"].format(
                topic=topic,
                detail=f"{topic}の重要な概念",
                benefit="効率的な問題解決"
            )
        elif "{text}" in template["prompt"]:
            prompt = template["prompt"].format(text=f"{topic}に関する詳細な説明")
            chosen = template["chosen"].format(
                point=f"{topic}の活用",
                detail="実用的な応用例"
            )
        else:
            prompt = template["prompt"].format(topic=topic)
            chosen = template["chosen"].format(
                topic=topic,
                definition="重要な技術分野",
                feature="高い効率性",
                usage="様々な業界"
            )
        
        dataset.append({
            "prompt": prompt,
            "chosen": chosen,
            "rejected": template["rejected"]
        })
    
    return dataset

# データセット生成
print("🔄 Generating DPO dataset...")
data = create_dpo_dataset(1000)
df = pd.DataFrame(data)

print(f"✅ Generated {len(df)} samples")
print(f"📊 Columns: {list(df.columns)}")
print("\n🔍 Sample data:")
display(df.head(2))

In [ ]:
# 🤖 モデルとトークナイザー設定
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

# Colab GPU環境に最適化された軽量モデル
model_name = "microsoft/DialoGPT-small"

print(f"📥 Loading model: {model_name}")

# トークナイザー読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# モデル読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

print("✅ Model and tokenizer loaded")
print(f"📊 Total parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# ⚡ LoRA設定（効率的な微調整）
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # Low rank for efficiency
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],
    bias="none"
)

# LoRAモデル作成
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

if torch.cuda.is_available():
    print(f"🔥 GPU memory usage: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

In [ ]:
# 📚 データ前処理
from datasets import Dataset
from trl import DPOTrainer, DPOConfig

# Dataset変換
dataset = Dataset.from_pandas(df)
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print(f"🎯 Training samples: {len(train_dataset)}")
print(f"🎯 Evaluation samples: {len(eval_dataset)}")

# サンプル確認
print("\n📝 Sample training data:")
print(train_dataset[0])

In [ ]:
# 🚀 DPOトレーニング設定と実行
training_args = DPOConfig(
    output_dir="./dpo_results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    fp16=True,
    gradient_checkpointing=True,
    max_length=128,
    max_prompt_length=64,
    report_to=["none"]  # Disable wandb
)

# DPOトレーナー初期化
dpo_trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    beta=0.1
)

print("⚙️ DPO Trainer initialized")
print("🔥 Starting training... This may take 10-15 minutes")

# トレーニング実行
train_result = dpo_trainer.train()

print("\n🎉 Training completed!")
print(f"📊 Final training loss: {train_result.training_loss:.4f}")

In [ ]:
# 📈 結果の評価と可視化
import matplotlib.pyplot as plt
import numpy as np

# 最終評価
eval_results = dpo_trainer.evaluate()
print("📊 Final Evaluation Results:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")

# トレーニング履歴の可視化
if hasattr(dpo_trainer.state, 'log_history'):
    log_history = dpo_trainer.state.log_history
    
    train_losses = []
    eval_losses = []
    steps = []
    
    for log in log_history:
        if 'loss' in log:
            train_losses.append(log['loss'])
            steps.append(log.get('step', len(train_losses)))
        if 'eval_loss' in log:
            eval_losses.append(log['eval_loss'])
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(steps, train_losses, 'b-', linewidth=2, label='Training Loss')
    plt.title('Training Loss Progress', fontsize=14)
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    if eval_losses:
        plt.subplot(1, 2, 2)
        eval_steps = np.linspace(0, max(steps), len(eval_losses))
        plt.plot(eval_steps, eval_losses, 'r-', linewidth=2, label='Evaluation Loss')
        plt.title('Evaluation Loss Progress', fontsize=14)
        plt.xlabel('Steps')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 🧪 トレーニング済みモデルのテスト
def generate_response(prompt, max_length=100):
    """DPOトレーニング済みモデルでレスポンス生成"""
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # プロンプト部分を除去
    if response.startswith(prompt):
        response = response[len(prompt):].strip()
    
    return response

# テスト実行
test_prompts = [
    "以下の質問に答えてください: 機械学習とは何ですか？",
    "次の文章を要約してください: プログラミングに関する詳細な説明",
    "プログラミングについて説明してください: Python"
]

print("🔬 Testing DPO-trained model...")
print("=" * 60)

for i, prompt in enumerate(test_prompts, 1):
    response = generate_response(prompt)
    print(f"\n📝 Test {i}:")
    print(f"Prompt: {prompt}")
    print(f"Response: {response}")
    print("-" * 40)

In [ ]:
# 💾 モデル保存とダウンロード
print("💾 Saving trained model...")

# モデル保存
dpo_trainer.save_model("./final_dpo_model")
tokenizer.save_pretrained("./final_dpo_model")

# ファイルサイズ確認
import os
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

model_size = get_folder_size('./final_dpo_model')
print(f"📊 Model size: {model_size / 1e6:.1f} MB")

# Google Driveに保存（オプション）
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    import shutil
    drive_path = '/content/drive/MyDrive/dpo_trained_model'
    shutil.copytree('./final_dpo_model', drive_path, dirs_exist_ok=True)
    print(f"☁️ Model saved to Google Drive: {drive_path}")

# ZIP形式でダウンロード準備
!zip -r dpo_model.zip ./final_dpo_model

print("\n🎉 DPO Training Successfully Completed!")
print("\n📋 Summary:")
print(f"   • Model: {model_name}")
print(f"   • Training samples: {len(train_dataset)}")
print(f"   • Evaluation samples: {len(eval_dataset)}")
print(f"   • Final training loss: {train_result.training_loss:.4f}")
print(f"   • Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

if torch.cuda.is_available():
    print(f"   • Peak GPU memory: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

print("\n📥 To download the model:")
print("   1. Download 'dpo_model.zip' from the file browser")
print("   2. Or access from Google Drive if mounted")